In [16]:
import os
import imp
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import sys
import pickle
import matplotlib.ticker as plticker
from scipy.signal import find_peaks

plt.rcParams["figure.figsize"] = (15,7)
from numpy.lib.scimath import sqrt as csqrt

#os.add_dll_directory('C:\\Program Files\\Lumerical\\v202\\api\\python\\')
#lumapi = imp.load_source('lumapi','C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py') 

sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
import lumapi
sys.path.append(os.path.dirname('/media/lucas/datam01/LucasLobo/BRAG-GRATINGS/simulations/python')) #Current directory

from jupyterthemes import jtplot

jtplot.style(theme='onedork', context='notebook', ticks=True, grid=True)
plt.rcParams["figure.figsize"] = (15,7)



def propagationmatrix(comprimento,neff,L): 
    #Comprimento = array comprimento de onda
    #neff = array neff em função do comprimento de onda
    #L = comprimento da seção uniforme da grade

    beta = 2*np.pi*np.divide(neff,comprimento)
    Tp = np.zeros([len(comprimento),2,2],dtype = 'complex_')
    Tp[:,0,0] = np.exp(1j*beta*L)
    Tp[:,1,1] = np.exp(-1j*beta*L)
    return Tp

def stepmatrix(n1,n2):
    #n1 = neff da grade de saída
    #n2 = neff da grade de entrada
    
    Ts = np.zeros([len(n1),2,2],dtype = 'complex_')
    Ts[:,0,0] = np.divide((n1+n2),(2*np.sqrt(np.multiply(n1+n2))))
    Ts[:,1,1] = np.divide((n1+n2),(2*np.sqrt(np.multiply(n1+n2))))
    Ts[:,0,1] = np.divide((n1-n2),(2*np.sqrt(np.multiply(n1+n2))))
    Ts[:,1,0] = np.divide((n1-n2),(2*np.sqrt(np.multiply(n1+n2))))
    return Ts

def TMM(comprimento,periodo,numerodegrades,n1,n2):
    #comprimento = array comprimento de onda
    #periodo = comprimento de bragg
    #numerodegrades = numero de periodos
    #n1 = neff da primeira parte da grade
    #n2 = neff da segunda parte da grade
    
    Tp1 = propagationmatrix(comprimento,n1,periodo/2)
    T12 = stepmatrix(n1,n2)
    Tp2 = propagationmatrix(comprimento,n2,periodo/2)
    T21 = stepmatrix(n2,n1)
    T = np.zeros([len(comprimento),2,2])
    Tmm = np.zeros([len(comprimento),2,2])
    for i in comprimento:
        T = Tp1[i,:,:]*T12[i,:,:]*Tp2[i,:,:]*T21[i,:,:]
    Tmm = T^numerodegrades
    return Tmm    

In [18]:
comprimento = [1000*1e-9,1500*1e-9,2000*1e-9]
neff = [3.1,3.2,3.3]
L = 10e-9
print(matrixHW(comprimento,neff,L))
print(np.sqrt(neff))

[[[0.98109052+0.19354947j 0.        +0.j        ]
  [0.        +0.j         0.98109052-0.19354947j]]

 [[0.99102991+0.13364026j 0.        +0.j        ]
  [0.        +0.j         0.99102991-0.13364026j]]

 [[0.99463081+0.10348695j 0.        +0.j        ]
  [0.        +0.j         0.99463081-0.10348695j]]]
[1.76068169 1.78885438 1.81659021]
